In [19]:
display("Adding style properties to notebook")
from IPython.core.display import HTML
HTML("""<style>div {
font-family:Open Sans,sans-serif;}
h1 {color:white;background-color: #446a7f;padding:10px}
h2{color:black; background-color:#b3b6a0; padding:10px}
h3{color:#78BE21; background-color:#446a7f; padding:10px}
h4{color:white; background-color:#78BE21;padding:5px}
.info{background-color:}
img {border: 3px solid #446a7f;}
</style>""")



# output directory for files created
outputdir = "./data/ruskin/output/json/"

images = {}

'Adding style properties to notebook'

# Linked Art - Transform - John Ruskin 

This notebook is transforms collections data for artworks by John Ruskin to a Linked Art representation, in preparation for a data visualisation. 


## Transformation Process
The transformation process common to all data sources:
- read collection data into Python dictionary
- create a mapping between entities in the Python dictionary and entities in the Linked Art data model
- use the mapping together with `createObjDesc()`, `cromulent` Python library, and custom coding in the `createObjProp()` function, to create a Linked Art JSON-LD representation
- write to file

In [1]:
try:
    import ipywidgets as widgets
except:
    !pip install ipywidgets
    import ipywidgets as widgets

from ipywidgets import Layout
from ipywidgets import FileUpload

import os


try:
    import IPython
except:
    !pip install IPython
    import IPython   
    
from IPython.display import display, IFrame, HTML

try:
    import xmltodict
except:
    !pip install xmltodict
    import xmltodict

try:
    import json
except:
    !pip install json
    import json 
    
    
try:
    import requests
except:
    !pip install requests
    import requests

import csv

try:
    import cromulent 
except:
    !pip install cromulent
    import cromulent
    
from cromulent.model import factory


from lib import linkedart as la



    
try:
    from ipy_table import *
except:
    !pip install ipy_table
    from ipy_table import *
    

    
# display python dictionary in HTML table
# adapated from https://bit.ly/3dVDy7v
def dict2table(d):

    # lists
    temp = []
    dictList = []

    # convert the dictionary to a list
    for key, value in d.items():
        temp = [key,value]
        dictList.append(temp)

    # create table with make_table
    make_table(dictList)
    
    # apply some styles to the table after it is created
    set_column_style(0, width='100', bold=True, color='hsla(225, 80%, 94%, 1)',align='left')
    set_column_style(1, width='100',align='left')

    # render the table
    return render()


## National Gallery of Art (NGA), Washington, United States

The collection data exists into two files:
- CSV data file containing artwork description  [Input data file](./data/nga/input/nga_ruskin.csv)
- CSV data file containing detailed digital image information for artworks https://raw.githubusercontent.com/NationalGalleryOfArt/opendata/main/data/published_images.csv


 ### Further Reading 
 
 - National Gallery of Art https://www.nga.gov/
 - NGA GitHub https://github.com/NationalGalleryOfArt

### Load NGA Collection Data into DataFrame

In [41]:
try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd
    
    
#  baseURI for JSON-LD document
baseURI = "https://www.nga.gov/collection/"

fileNGA = 'data/nga/input/nga_ruskin.csv'

dataFrameNGA = pd.read_csv(fileNGA)
dataFrameNGA.head()

,objectid,accessioned,accessionnum,locationid,title,displaydate,beginyear,endyear,visualbrowsertimespan,medium,...,visualbrowserclassification,parentid,isvirtual,departmentabbr,portfolio,series,volume,watermarks,lastdetectedmodification,customprinturl
0,70238,1,1987.73.2,NaN,Tower of the Cathedral at Sens,c. 1845,1845,1845,1826 to 1850,"pen and brown ink, brush and black ink, black ...",...,drawing,NaN,0,CG-E,NaN,NaN,NaN,NaN,2019-10-28 22:01:34.883-04,NaN
1,70367,1,1988.20.38,NaN,Tree Study,mid-1850s,1845,1855,1826 to 1850,pen and black ink with blue-gray and gray wash...,...,drawing,NaN,0,CG-E,NaN,NaN,NaN,NaN,2020-04-10 22:01:40.093-04,NaN
2,72870,1,1991.88.1,NaN,The Garden of San Miniato near Florence,1845,1845,1845,1826 to 1850,"watercolor and pen and black ink, heightened w...",...,drawing,NaN,0,CG-E,NaN,NaN,NaN,NaN,2019-10-28 22:01:34.883-04,NaN
3,76140,1,1995.52.158,NaN,"Ornamental Study with Acanthus Motif for ""The ...",1849,1849,1849,1826 to 1850,pen and brown ink with watercolor and graphite...,...,drawing,NaN,0,CG-E,NaN,NaN,NaN,NaN,2019-10-28 22:01:34.883-04,NaN


### Load NGA Digital Image File into DataFrame

The data file containing detailed digital image data is loaded into a pandas dataframe `dataFrameNGAImages`

In [43]:
fileNGAimages = "https://raw.githubusercontent.com/NationalGalleryOfArt/opendata/main/data/published_images.csv"
dataFrameNGAImages = pd.read_csv(fileNGAimages)
dataFrameNGAImages.head()

,uuid,iiifurl,iiifthumburl,viewtype,sequence,width,height,maxpixels,created,modified,depictstmsobjectid,assistivetext
0,f62208e4-c1b6-4dfb-a84c-77645a2c9f77,https://api.nga.gov/iiif/f62208e4-c1b6-4dfb-a8...,https://api.nga.gov/iiif/f62208e4-c1b6-4dfb-a8...,primary,0.0,2863,4000,640.0,2013-10-29 10:50:11-04,2021-09-03 12:55:55-04,163048,NaN
1,f62286ed-8688-4dca-8452-e629a48ca8ec,https://api.nga.gov/iiif/f62286ed-8688-4dca-84...,https://api.nga.gov/iiif/f62286ed-8688-4dca-84...,primary,0.0,3423,4322,NaN,2014-04-17 15:41:39-04,2021-07-07 17:39:52-04,19085,NaN
2,f6228db8-3e07-4590-b30f-40030b4c9aa9,https://api.nga.gov/iiif/f6228db8-3e07-4590-b3...,https://api.nga.gov/iiif/f6228db8-3e07-4590-b3...,primary,0.0,4410,3476,NaN,2014-07-07 15:05:14-04,2021-07-07 18:01:50-04,16038,NaN
3,f622a93d-ed29-4305-8845-f0b1469c8932,https://api.nga.gov/iiif/f622a93d-ed29-4305-88...,https://api.nga.gov/iiif/f622a93d-ed29-4305-88...,primary,0.0,4507,4507,NaN,2017-10-06 15:43:11-04,2021-08-03 14:24:21-04,110736,NaN
4,f6254acb-348e-46a1-b92c-8d8acc97bf28,https://api.nga.gov/iiif/f6254acb-348e-46a1-b9...,https://api.nga.gov/iiif/f6254acb-348e-46a1-b9...,primary,0.0,4000,2692,NaN,2015-04-22 10:14:29-04,2021-07-08 17:35:41-04,32527,NaN


### Remove Byte Order Marks and Create Data Mapping

Remove Byte Order Marks and create Python dictionary containing data mapping for each input file.

In [70]:
import pandas as pd


# remove BOM

removeBOM = open(fileNGA, mode='r', encoding='utf-8-sig').read()
open(fileNGA, mode='w', encoding='utf-8').write(removeBOM)


# data mapping between Linked Art data model and NGA data entities
mapp_nga =  {
    "id":"objectid",
    "accession_number":"accessionnum",
    "accession_date": "",
    "classification" : "classification",
    "title": "title",
    "alt_title": "",
    "notes": "",
    "date_created":"displaydate",
    "date_created_earliest": "beginyear",
    "date_created_latest": "endyear",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "created_provenance" : "",
    "creator":"attribution",
    "physical_medium": "medium",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "dimensions",
    "created_provenance": "" ,
    "credit_line": "creditline",
    "collection" : "departmentabbr",
    "current_status" : "",
    "current_owner" : "",
    "homepage": ""
}

dict2table(mapp_nga)

#df = pd.DataFrame(mapp_nga, index=[0])
#display(df) 

    

id,objectid
accession_number,accessionnum
accession_date,
classification,classification
title,title
alt_title,
notes,
date_created,displaydate
date_created_earliest,beginyear
date_created_latest,endyear
created_period,


In [77]:


dfimg_list = dataFrameNGAImages['depictstmsobjectid'].tolist()
dfimgurl_list = dataFrameNGAImages['iiifurl'].tolist()

fileNGA = 'data/nga/input/nga_ruskin.csv'

def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creator":
                    objProp[prop] = [{"id": baseURI +"creatorid/" + obj[mapp["id"]] ,"name": obj[key],"role":"Artist"}]
                else:
                    objProp[prop] = obj[key]
    for image in images:
        if objProp["id"] in image:
            objProp["image_url"] = images[image]
    return objProp   

    
allObjects = csv.DictReader(open(fileNGA, mode='r',encoding='utf-8'))

for obj in allObjects:
    # create object property dictionary
    objProp = createObjProp(obj,mapp_nga)
    id = objProp["id"]

    if int(id) in dfimg_list:
        df_images_match = df_images.loc[df_images['depictstmsobjectid'] == int(id)]
        objProp["image_url"] = df_images_match.iloc[0]["iiifurl"] + "/full/!500,500/0/default.jpg"
        
    objProp["homepage"] = "https://www.nga.gov/collection/art-object-page." + id + ".html"
    object_uri = baseURI + id
    
    # create obj description
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)

    # write to file 
    text_file = open(outputdir + objProp["id"] + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()
    
    display(pd.DataFrame(objProp, index=[0]))
   
    



,id,accession_number,title,date_created,date_created_earliest,date_created_latest,physical_medium,physical_dimensions,creator,credit_line,classification,collection,image_url,homepage
0,70238,1987.73.2,Tower of the Cathedral at Sens,c. 1845,1845,1845,"pen and brown ink, brush and black ink, black ...",sheet: 40.7 x 30.6 cm (16 x 12 1/16 in.),{'id': 'https://www.nga.gov/collection/creator...,Gift of William B. O'Neal,Drawing,CG-E,https://api.nga.gov/iiif/49a6128c-8d5a-4b00-be...,https://www.nga.gov/collection/art-object-page...


,id,accession_number,title,date_created,date_created_earliest,date_created_latest,physical_medium,physical_dimensions,creator,credit_line,classification,collection,image_url,homepage
0,70367,1988.20.38,Tree Study,mid-1850s,1845,1855,pen and black ink with blue-gray and gray wash...,overall: 28.4 x 38.1 cm (11 3/16 x 15 in.),{'id': 'https://www.nga.gov/collection/creator...,Gift of John Nichols Estabrook and Dorothy Coo...,Drawing,CG-E,https://api.nga.gov/iiif/9310c903-7099-4138-b3...,https://www.nga.gov/collection/art-object-page...


,id,accession_number,title,date_created,date_created_earliest,date_created_latest,physical_medium,physical_dimensions,creator,credit_line,classification,collection,image_url,homepage
0,72870,1991.88.1,The Garden of San Miniato near Florence,1845,1845,1845,"watercolor and pen and black ink, heightened w...",Overall: 34.2 x 49.2 cm (13 7/16 x 19 3/8 in.)...,{'id': 'https://www.nga.gov/collection/creator...,Patrons' Permanent Fund,Drawing,CG-E,https://api.nga.gov/iiif/f6ef48d3-3512-4f46-ac...,https://www.nga.gov/collection/art-object-page...


,id,accession_number,title,date_created,date_created_earliest,date_created_latest,physical_medium,physical_dimensions,creator,credit_line,classification,collection,image_url,homepage
0,76140,1995.52.158,"Ornamental Study with Acanthus Motif for ""The ...",1849,1849,1849,pen and brown ink with watercolor and graphite...,overall: 20.1 x 16.6 cm (7 15/16 x 6 9/16 in.),{'id': 'https://www.nga.gov/collection/creator...,Gift of William B. O'Neal,Drawing,CG-E,https://api.nga.gov/iiif/bc567179-9c1e-4493-b7...,https://www.nga.gov/collection/art-object-page...


### Cleveland Museum of Art 
http://www.clevelandart.org/
- Download CSV file from GitHub https://github.com/ClevelandMuseumArt/openaccess
- View file with OpenRefine https://openrefine.org/
- Create a text facet for the Creator field to identify artworks by John Ruskin
- Export this subset of records (1 record found)
- [data file](./data/cma/input/ruskin.csv) 


In [83]:
#  baseURI for JSON-LD document
baseURI = "https://clevelandart.org/art/"
file = './data/cma/input/ruskin.csv'
mpg = pd.read_csv(file)
mpg.head()

,id,accession_number,share_license_status,tombstone,current_location,title,title_in_original_language,series,series_in_original_language,creation_date,...,digital_description,wall_description,external_resources,citations,catalogue_raisonne,url,image_web,image_print,image_full,updated_at
0,154494,1989.14,CC0,"Budding Sycamore, c. 1876. John Ruskin (Britis...",NaN,Budding Sycamore,NaN,NaN,NaN,c. 1876,...,The most influential art critic in Britain dur...,The most influential art critic in Britain dur...,"{'wikidata': [], 'internet_archive': []}","Cook, Edward Tyas and Alexander Wedderburn. <e...",NaN,https://clevelandart.org/art/1989.14,https://openaccess-cdn.clevelandart.org/1989.1...,https://openaccess-cdn.clevelandart.org/1989.1...,https://openaccess-cdn.clevelandart.org/1989.1...,2021-03-27 12:16:10.046000


In [84]:

#remove BOM
s = open(file, mode='r', encoding='utf-8-sig').read()
open(file, mode='w', encoding='utf-8').write(s)

mapp_cma =  {
    "id":"id",
    "accession_number":"accession_number",
    "accession_date": "",
    "classification" : "type",
    "title": "title",
    "alt_title": "title_in_original_language",
    "notes": "tombstone",
    "date_created":"creation_date",
    "date_created_earliest": "creation_date_earliest",
    "date_created_latest": "creation_date_latest",
    "created_period":"culture",
    "created_dynasty":"",
    "created_inscriptions":"inscriptions",
    "created_notes": "fun_fact",
    "creator":"creator",
    "physical_medium": "Medium",
    "physical_style": "",
    "physical_technique": "technique",
    "physical_description": "",
    "physical_dimensions": "measurements",
    "created_provenance": "provenance" ,
    "credit_line": "creditline",
    "collection" : "department",
    "current_status" : "current_location",
    "current_owner": "",
    "creator": {"name": "John Ruskin"},
    "homepage": "url"
    
    
}

dict2table(mapp_cma)

id,id
accession_number,accession_number
accession_date,
classification,type
title,title
alt_title,title_in_original_language
notes,tombstone
date_created,creation_date
date_created_earliest,creation_date_earliest
date_created_latest,creation_date_latest
created_period,culture


In [85]:


def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creator":
                    objProp[prop] = [{"id": baseURI +"creatorid/" + obj[mapp["id"]] ,"name": obj[key],"role":"Artist"}]
                else:
                    objProp[prop] = obj[key]
    return objProp   

allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))

for obj in allObjects:
    # create object property dictionary
    objProp = createObjProp(obj,mapp_cma)

    id = objProp["id"]
    object_uri = baseURI + id
    
    objProp["image_url"] = "https://clevelandart.org/art/" + id
    
    # create obj description
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
    
    # write to file 
    text_file = open(outputdir + "/" + objProp["id"] + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()
    display(pd.DataFrame(objProp, index=[0]))
    
    

,id,accession_number,notes,current_status,title,alt_title,date_created,date_created_earliest,date_created_latest,created_period,physical_technique,collection,classification,physical_dimensions,credit_line,created_inscriptions,created_provenance,created_notes,homepage,image_url
0,154494,1989.14,"Budding Sycamore, c. 1876. John Ruskin (Britis...",,Budding Sycamore,,c. 1876,1871,1881,"England, 19th century","black and gray wash, gouache, and graphite",Drawings,Drawing,Sheet: 34.8 x 44.8 cm (13 11/16 x 17 5/8 in.),Andrew R. and Martha Holden Jennings Fund,"signed and inscribed, in brown ink, at lower r...","G.R. Roxburgh (?-?); (Artemis Fine Arts, Londo...",This drawing was used as an illustration in Jo...,https://clevelandart.org/art/1989.14,https://clevelandart.org/art/154494


## Tate Museum

The Tate Museum has three items by John Ruskin in its Collection Online database. JSON format file are available from 

https://github.com/tategallery/collection

### N02726 

https://www.tate.org.uk/art/artworks/ruskin-an-olive-spray-and-two-leaf-outlines-n02726 

https://raw.githubusercontent.com/tategallery/collection/master/artworks/n/027/n02726-13031.json 

 

### N02972 

https://www.tate.org.uk/art/artworks/ruskin-the-north-west-angle-of-the-facade-of-st-marks-venice-n02972 

https://raw.githubusercontent.com/tategallery/collection/master/artworks/n/029/n02972-13032.json  

 
### N03507 

https://www.tate.org.uk/art/artworks/ruskin-view-of-bologna-n03507 

https://raw.githubusercontent.com/tategallery/collection/master/artworks/n/035/n03507-13033.json  

In [86]:
fileprefix = "./data/tate/"

files = ("n02726-13031.json", 
         "n02972-13032.json", 
         "n03507-13033.json")

baseURI = "https://www.tate.org.uk/art/artworks/"

mapp_tate =  {
    "id":"id",
    "accession_number":"acno",
    "accession_date": "acquisitionYear",
    "classification" : "classification",
    "title": "title",
    "alt_title": "",
    "notes": "",
    "date_created":"dateText",
    "date_created_earliest": "Dated",
    "date_created_latest": "Dated",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"contributors",
    "physical_medium": "medium",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "dimensions",
    "created_provenance": "" ,
    "credit_line": "creditLine",
    "collection" : "Department",
    "current_status" : "",
    "current_owner": "Tate Museum",
    "image_url":"thumbnailUrl",
    "homepage" : "url"
}

dict2table(mapp_tate)

id,id
accession_number,acno
accession_date,acquisitionYear
classification,classification
title,title
alt_title,
notes,
date_created,dateText
date_created_earliest,Dated
date_created_latest,Dated
created_period,


In [88]:



def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creator":
                    objProp[prop] = [{"name": obj[key]}]
                else:
                    objProp[prop] = obj[key]
    return objProp   

for file in files:
    objProp = {}
    with open(fileprefix + file) as json_file:
        data = json.load(json_file)
        objProp =  {
            "id": data["id"],
            "accession_number":data["acno"],
            "accession_date": data["acquisitionYear"],
            "classification" : data["classification"],
            "title": data["title"],
            "alt_title": "",
            "notes": "",
            "date_created":data["dateText"],
            "date_created_earliest": "",
            "date_created_latest": "",
            "created_period":"",
            "created_dynasty":"",
            "created_inscriptions":"",
            "created_notes": "",
            "creator":[],
            "physical_medium": data["medium"],
            "physical_style": "",
            "physical_technique": "",
            "physical_description": "",
            "physical_dimensions": data["dimensions"],
            "created_provenance": "" ,
            "credit_line": data["creditLine"],
            "collection" : "",
            "current_status" : "",
            "current_owner": "Tate Museum",
            "image_url" : "thumbnailUrl",
            "homepage" : "url"
            }
        
        creator = data["contributors"]
        for contributor in creator:
            id = str(contributor["id"])
            name = str(contributor["fc"])
            objProp["creator"].append({"id":id,"name":name})
            
        objProp["current_owner"] = {"name":"Tate Museum",
                                "location":"London,England",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
        id = str(objProp["id"])
        object_uri = baseURI + str(id)
    
        # create obj description
        objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
     
        # write to file 
        text_file = open(outputdir + "/" + id + ".json", "wt")
        n = text_file.write(factory.toString(objLA, compact=False))
        text_file.close()
        display(pd.DataFrame(objProp, index=[0]))
        
#dict2table(objProp)        

,id,accession_number,accession_date,classification,title,alt_title,notes,date_created,date_created_earliest,date_created_latest,...,physical_technique,physical_description,physical_dimensions,created_provenance,credit_line,collection,current_status,current_owner,image_url,homepage
0,13031,N02726,1910,"on paper, unique",An Olive Spray and Two Leaf Outlines,,,before 1877,,,...,,,support: 375 x 279 mm,,Presented by Sir Claude Phillips in memory of ...,,,NaN,thumbnailUrl,url


,id,accession_number,accession_date,classification,title,alt_title,notes,date_created,date_created_earliest,date_created_latest,...,physical_technique,physical_description,physical_dimensions,created_provenance,credit_line,collection,current_status,current_owner,image_url,homepage
0,13032,N02972,1914,"on paper, unique",The North-West Angle of the Facade of St Mark’...,,,date not known,,,...,,,support: 940 x 610 mm,,Presented by the Art Fund 1914,,,NaN,thumbnailUrl,url


,id,accession_number,accession_date,classification,title,alt_title,notes,date_created,date_created_earliest,date_created_latest,...,physical_technique,physical_description,physical_dimensions,created_provenance,credit_line,collection,current_status,current_owner,image_url,homepage
0,13033,N03507,1920,"on paper, unique",View of Bologna,,,c.1845–6,,,...,,,support: 343 x 489 mm,,Purchased 1920,,,NaN,thumbnailUrl,url


## Harvard Art Museum

https://harvardartmuseums.org/collections/person/28419?person=28419


In [89]:
apidocs = "https://github.com/harvardartmuseums/api-docs"

keyfile = "data/ruskin/input/key.txt"
key = open(keyfile, mode='r', encoding='utf-8-sig').read()
uri = "https://api.harvardartmuseums.org/object?person=28419&apikey=" + key

for page in range(1, 11):
    response = requests.get(uri + "&page=" + str(page))
    json_data = response.json()
    text_file = open( "./data/harvard/input/" + str(page) + ".json", "wt")
    n = text_file.write(json.dumps(json_data, indent=2))
    text_file.close()


### mapping

In [90]:
import re

baseURI = "https://www.harvardartmuseums.org/collections/object/"

mapp_harvard =  {
    "id":"id",
    "accession_number":"objectnumber",
    "accession_date": "accessionyear",
    "classification" : "",
    "title": "title",
    "alt_title": "",
    "notes": "commentary",
    "date_created":"dated",
    "date_created_earliest": "",
    "date_created_latest": "",
    "created_period":"period",
    "created_dynasty":"century",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"people",
    "physical_medium": "medium",
    "physical_style": "",
    "physical_technique": "technique",
    "physical_description": "description",
    "physical_dimensions": "dimensions",
    "created_provenance": "provenance" ,
    "credit_line": "creditline",
    "collection" : "division",
    "classification": "classification",
    "current_status" : "",
    "current_owner": "Harvard Museum of Art",
    "image_url":"primaryimageurl",
    "homepage" : "url"
}

dict2table(mapp_harvard)

id,id
accession_number,objectnumber
accession_date,accessionyear
classification,classification
title,title
alt_title,
notes,commentary
date_created,dated
date_created_earliest,
date_created_latest,
created_period,period


In [92]:


def createObjPropHarvard(obj,mapp):

    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if key == "dated":
                    
                    # look for year in value 
                    years = re.findall('(\d{4})', obj[key])
                    if len(years) == 1:
                        #objProp["date"] = years[0]
                        objProp["date_created_earliest"] = years[0]
                        objProp["date_created_latest"] = years[0]
                    elif len(years) == 2:
                       # objProp["dated"] = str(years[0]) + "-" + str(years[1])
                        objProp["date_created_earliest"] = years[0]
                        objProp["date_created_latest"] = years[1]
                    else:
                        # use Ruskins birth/death years
                        objProp["date_created_earliest"] = 1819
                        objProp["date_created_latest"] = 1900
                        
    
                        
                        
                
                if prop == "creator":
                    objProp["creator"] = ""
                    for person in obj[key]:
                        
                        if "personid" in person and person["personid"] == 28419 and person["role"] == "Artist": 
                            objProp[prop] = [{
                                "id": str(person["personid"]),
                                "name": person["displayname"],
                                }]      
                else:
                    objProp[prop] = obj[key]
    objProp["current_owner"] = {"name":"Harvard Art Museum",
                                "location":"Cambridge, MA",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
    return objProp   

for x in range(1,11):
    
    with open("./data/harvard/input/" + str(x) + ".json") as json_file:
        data = json.load(json_file)
        for obj in data["records"]:
            #print(json.dumps(obj,indent=2))
            objProp=createObjPropHarvard(obj,mapp_harvard)
            if "image_url" in objProp and objProp["image_url"] != "":
                images["https://www.harvardartmuseums.org/collections/object/" + str(objProp["id"])] = objProp["image_url"]
            if objProp["creator"] != "":
                #print(json.dumps(objProp,indent=2))
                
                id = str(objProp["id"])
                object_uri = baseURI + id
    
                # create obj description
                objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
                #print(json.dumps(objLA,indent=2))
                # write to file 
                text_file = open(outputdir + "/" + id + ".json", "wt")
                #print(factory.toString(objLA, compact=False))
                n = text_file.write(factory.toString(objLA, compact=False))
                text_file.close()
                display(pd.DataFrame(objProp, index=[0]))
    
    
#dict2table(objProp)   



,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Gift of Miss ...",European and American Art,262409,None,1921,Prints,https://nrs.harvard.edu/urn-3:HUAM:INV194168_d...,None,Etching,None,...,None,1819,1900,19th century,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,M2084,None,NaN


,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Bequest of Gr...",European and American Art,262414,None,1943,Prints,https://nrs.harvard.edu/urn-3:HUAM:INV194145_d...,None,Mezzotint,None,...,None,1819,1900,19th century,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,M10900,None,NaN


,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Gift of Miss ...",European and American Art,262415,None,1921,Prints,https://nrs.harvard.edu/urn-3:HUAM:INV194153_d...,None,Etching and mezzotint,None,...,None,1819,1900,19th century,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,M2086,None,NaN


,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Gift of Dr. D...",European and American Art,262444,None,1919,Prints,https://nrs.harvard.edu/urn-3:HUAM:INV194183_d...,None,Etching,None,...,None,1819,1900,19th century,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,M962,plate: 20.6 x 28.6 cm (8 1/8 x 11 1/4 in.),NaN


,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Bequest of Mr...",European and American Art,296290,None,1970,Drawings,https://nrs.harvard.edu/urn-3:HUAM:779025,None,None,None,...,The artist; to William Ward II (1829-1908). A...,1874,1874,c. 1874,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,1970.74,20.9 x 16.1 cm (8 1/4 x 6 5/16 in.),NaN


...

,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Gift of Miss ...",European and American Art,262445,None,1921,Prints,https://nrs.harvard.edu/urn-3:HUAM:INV194147_d...,None,Etching,None,...,None,1819,1900,19th century,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,M2122,None,NaN


,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Gift of Miss ...",European and American Art,262999,None,1921,Prints,https://nrs.harvard.edu/urn-3:HUAM:INV194193_d...,None,Etching,None,...,None,1819,1900,19th century,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,M2083,None,NaN


,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Gift of Meta ...",European and American Art,297450,None,1949,Drawings,https://nrs.harvard.edu/urn-3:HUAM:VRS46314_dynmc,None,None,None,...,None,1863,1869,c. 1863-1869,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,1949.30.A,226 x 173 cm (89 x 68 1/8 in.),NaN


,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Transfer from...",European and American Art,298364,None,1926,Drawings,https://nrs.harvard.edu/urn-3:HUAM:70390_dynmc,None,None,None,...,None,1819,1900,19th century,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,1926.33.192,16.3 x 22 cm (6 7/16 x 8 11/16 in.),NaN


,credit_line,collection,id,created_period,accession_date,classification,image_url,notes,physical_technique,physical_description,...,created_provenance,date_created_earliest,date_created_latest,date_created,creator,homepage,created_dynasty,accession_number,physical_dimensions,current_owner
0,"Harvard Art Museums/Fogg Museum, Gift of Samue...",European and American Art,298628,None,1919,Drawings,https://nrs.harvard.edu/urn-3:HUAM:38491_dynmc,"A portion of this drawing is engraved, in reve...",None,None,...,"Charles Eliot Norton, to; his daughters Sara, ...",1848,1848,1848,"{'id': '28419', 'name': 'John Ruskin'}",https://www.harvardartmuseums.org/collections/...,19th century,1919.504,48.1 x 36.2 cm (18 15/16 x 14 1/4 in.),NaN


## Rijksmuseum

In [94]:
keyfile = "data/ruskin/input/keyrijks.txt"
key = open(keyfile, mode='r', encoding='utf-8-sig').read()
url = "https://www.rijksmuseum.nl/api/nl/collection?key=" + key + "&involvedMaker=John+Ruskin"

rijksdir = "data/rijks/json/"

baseURI = "http://www.rijksmuseum.nl/nl/collectie/"


response = requests.get(url)
data = response.json()

for artwork in data["artObjects"]:
    
    if artwork["hasImage"] == False:
        continue
    if artwork["principalOrFirstMaker"] != "John Ruskin":
        continue
    text_file = open( rijksdir + artwork["id"] + ".json", "wt")
    text_file.write(json.dumps(artwork, indent=2))
    text_file.close()
    
    
    
mapp = {
    
    "id":"id",
    "accession_number":"objectNumber",
    "classification" : "",
    "title": "title",
    "alt_title": "longTitle",
    "notes": "",
    "date_created":"",
    "date_created_earliest": "",
    "date_created_latest": "",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"principalOrFirstMaker",
    "physical_medium": "",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "",
    "created_provenance": "" ,
    "credit_line": "",
    "collection" : "",
    "classification": "",
    "current_status" : "",
    "current_owner": "",
    "image_url":"",
    "homepage" : ""
}

dict2table(mapp)

id,id
accession_number,objectNumber
classification,
title,title
alt_title,longTitle
notes,
date_created,
date_created_earliest,
date_created_latest,
created_period,
created_dynasty,


In [95]:

def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creator":
                    objProp["creator"] = ""
                    for person in obj[key]:
                        
                        if "personid" in person and person["personid"] == 28419 and person["role"] == "Artist": 
                            objProp[prop] = [{
                                "id": str(person["personid"]),
                                "name": person["displayname"],
                                }]      
                else:
                    objProp[prop] = obj[key]
    objProp["current_owner"] = {"name":"Rijksmuseum",
                                "location":"Amsterdam, Netherlands",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": "Museum"}
    return objProp   


file_list=os.listdir(rijksdir)
for file in file_list:
    with open(rijksdir + file) as json_file:
        obj = json.load(json_file)
        objProp=createObjProp(obj,mapp)
        
        objProp["creator"] = obj["principalOrFirstMaker"]
        objProp["classification"] = "Painting"
        objProp["homepage"] = obj["links"]["web"]
        
        
        
        if objProp["creator"] != "":
           
            id = str(objProp["id"])
            
            object_uri = baseURI + id
            
            # create obj description
            objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
            # write to file 
            text_file = open(outputdir + id + ".json", "wt")
            n = text_file.write(factory.toString(objLA, compact=False))
            text_file.close()
            display(pd.DataFrame(objProp, index=[0]))

#dict2table(objProp)



,id,accession_number,title,creator,alt_title,current_owner,classification,homepage
0,nl-RP-T-1987-19,RP-T-1987-19,Bergkam aan het Meer van Genève,John Ruskin,"Bergkam aan het Meer van Genève, John Ruskin, ...",NaN,Painting,http://www.rijksmuseum.nl/nl/collectie/RP-T-19...


,id,accession_number,title,creator,alt_title,current_owner,classification,homepage
0,nl-RP-T-1957-335,RP-T-1957-335,"Gezicht op S. Anastasia te Verona, over de Adige",John Ruskin,"Gezicht op S. Anastasia te Verona, over de Adi...",NaN,Painting,http://www.rijksmuseum.nl/nl/collectie/RP-T-19...


## Ashmolean Museum

In [96]:
import requests

try:
    from bs4 import BeautifulSoup
except:
    !pip install bs4
    from bs4 import BeautifulSoup
    
import re

from cromulent.model import factory
    
from lib import linkedart as la


baseURI = "https://collections.ashmolean.org/collection/"


ash_file = '/Users/tanya/Documents/Code/linkedart/linked-art/data/ashmolean/ash.html'

html_text = open(ash_file, 'r').read()
soup = BeautifulSoup(html_text, 'html.parser')




for artwork in soup.find_all('div',attrs={"class":"list-inner"}):
    
    title = artwork.find('h3').string
    id = artwork.find('h3').find('a').get('href')
    id = id[-6:]
    creator = [{"id": "", "name":"John Ruskin","role":"Artist"}]
    accession_number = artwork.find('span').string
    accession_number = accession_number.replace("Accession no. ", "")
    image_url = artwork.find("img").get("src")
    date_created = artwork.find("p").string
    date_created = date_created.replace("Date: ","")
    
    objProp = {
        "classification":"",
        "homepage":"https://collections.ashmolean.org/collection/browse-9148/object/" + id,
        "id":id,
        "title":title,
        "creator":creator,
        "accession_number":accession_number,
        "image_url":image_url,
        "date_created":date_created,
        "current_owner" : {"name":"Ashmolean Museum",
                                "location":"Oxford, England, UK",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
    }
    
    # look for year in value 
    years = re.findall('(\d{4})', date_created)
    if len(years) == 1:
                        
        objProp["date_created_earliest"] = years[0]
        objProp["date_created_latest"] = years[0]
    elif len(years) == 2:                
        objProp["date_created_earliest"] = years[0]
        objProp["date_created_latest"] = years[1]
    else:
        # use Ruskins birth/death years
        objProp["date_created_earliest"] = 1819
        objProp["date_created_latest"] = 1900
    
    
    object_uri = baseURI + id
    
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
    
    # write to file 
    text_file = open(outputdir + objProp["id"] + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()
    
dict2table(objProp)

classification,
homepage,https://collections.ashmolean.org/collection/browse-9148/object/103910
id,103910
title,Small sketch stated to be by John Ruskin
creator,"[{'id': '', 'name': 'John Ruskin', 'role': 'Artist'}]"
accession_number,WA1960.77.47.4
image_url,https://collections.ashmolean.org/static/shared/assets/placeholder_collection_w800.png
date_created,possibly 1870 - 1880
current_owner,"{'name': 'Ashmolean Museum', 'location': 'Oxford, England, UK', 'type': 'http://vocab.getty.edu/aat/300312281', 'type_label': ''}"
date_created_earliest,1870
date_created_latest,1880


## Result - Linked Art JSON-LD Files for John Ruskin Artworks

The result of the transformation is a collection of Linked Art JSON-LD files for artworks by John Ruskin from various collections.

You can explore the Linked Art representation below in the JSON-LD visualisation:
- select an artwork from the dropdown menu to view the image and associated Linked Art JSON-LD visualisation

In [101]:
file_list=os.listdir(outputdir)
   
#for file in file_list:
  #  display(HTML("<a target='_new' href='" + outputdir + file +"'>" + file + "</a>"))


In [102]:
import ipywidgets
from ipywidgets import Layout, FileUpload 
from IPython.display import display, IFrame, HTML, Image
import os
import json
from IPython.core.display import Javascript        


def dropdown_eventhandler(change):
    with open('./src/js/visld.js', 'r') as _jscript:
        code = _jscript.read() + "var file = '" + outputdir + change.new + "';var selector = '#vis3';visjsonld(file, selector); "
        display(Javascript(code))
    
        with open( dir + "/" + change.new) as json_file:
    
                artwork = json.load(json_file)
                if ("representation" in artwork):
                    image = artwork["representation"][0]["id"]
                    display(Javascript("document.getElementById('artwork').src = '" + image   +   "';"))
                else:
                    display(Javascript("document.getElementById('artwork').src = '';"))
                    

# directory that will contain updated Ruskin artwork representations including geo coords
dir = outputdir = "./data/ruskin/output/json/"

file_list=os.listdir(dir)

selectOptions = []
selectOptions.append(("Please select an artwork", ""))


# for each linked art json file
for file in file_list:
    # open file
    with open( dir + "/" + file) as json_file:
        artwork = json.load(json_file)  
        if artwork.get('_label') != None:
            title = artwork["_label"] + " (" + file + ")"
        else:
            title = " (" + file + ")"
       
        selectOptions.append((title,file))

In [103]:
selectObject = ipywidgets.Dropdown(options=selectOptions)
selectObject.observe(dropdown_eventhandler, names='value')

display(selectObject)

Dropdown(options=(('Please select an artwork', ''), ("Engraving of Ruskin's Drawing of the Petal Vault of a Sc…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<div><img style="height:500px" id="artwork" src=""/></div>

<div id='vis3' style='height:100%;width:6000px'></div>